In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
# build the computational graph tf.graph
# run the computational graph tf.session

# tf.operations - calculations that consumes and produces tensors
# tf.tensors - the edges in the graph, values that flow through the graph


a = tf.constant(3.0, dtype = tf.float32)
b = tf.constant(4.0, dtype = tf.float32)

total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [3]:
# save the computational graph to a tensorboard summary file
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [5]:
sess = tf.Session()
print (sess.run(total))

7.0


In [6]:
print(sess.run({'ab':(a,b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [7]:
vec = tf.random_uniform(shape= (3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[ 0.31754375  0.90904057  0.87698412]
[ 0.49251163  0.37676251  0.32501101]
(array([ 1.39655161,  1.04024816,  1.95503247], dtype=float32), array([ 2.39655161,  2.04024816,  2.95503235], dtype=float32))


In [8]:
# a graph can be parameterized to accept external inputs, known as placeholders
# a placeholder is a promise to provide a value later, like a function argument

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

print(sess.run(z, feed_dict = {x:3, y:4.5}))
print(sess.run(z, feed_dict = {x:[1,3], y:[2,4]}))

7.5
[ 3.  7.]


In [9]:
# the only difference between placeholders and other tf.Tensors is that placeholders
# throw an error if no value is fed to them

# placeholders work for simple experiments, but tf.data are the preferred method
# of streaming data into a model

my_data = [
    [0,1],
    [2,3],
    [4,5],
    [6,7],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [10]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [15]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

[-1.05413342 -1.42547834  1.70376122]
[-1.3831712   0.138587    0.60081166]
[ 0.06026712  0.84539485 -1.63404822]
[-0.39682257 -0.20598796  0.27671582]
[ 0.69473881  0.97860539 -1.80610359]
[-0.13793366 -0.00392134  0.91621238]
[-0.25455409  1.75833642 -0.53085917]
[-0.37180507  0.45843273 -1.55250537]
[-1.82615149 -0.48195559  0.1630346 ]
[-0.54157573  0.22197683  0.87383032]


In [16]:
print(sess.run(r))

[[ 0.2716502   0.97716582  1.07672477]
 [-0.05653711  1.48598814 -1.5080626 ]
 [ 0.46906993 -0.56370842  0.90755999]
 [-0.44586554  0.94789994 -1.47499228]
 [-0.93399984 -1.87921381  0.8691389 ]
 [ 0.90690857 -0.13226394  0.7703138 ]
 [ 0.46517599 -0.46254516  0.30981454]
 [-2.40581703  0.80254656  0.31266794]
 [-0.1715571   0.25611329 -0.4901334 ]
 [ 0.57785183 -0.22973686  0.50004947]]


In [18]:
# creating layers

x = tf.placeholder(tf.float32, shape = [None, 3])
linear_model = tf.layers.Dense(units = 1)
y = linear_model(x)

In [19]:
# intializing all variables in a tensorflow graph
init = tf.global_variables_initializer()
sess.run(init)

In [20]:
# Executing layers
print(sess.run(y, {x:[[1,2,3],[4,5,6]]}))

[[-3.86432433]
 [-6.7078805 ]]


In [21]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [22]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [23]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [24]:
print(sess.run(inputs))

[[  1.   0.   5.]
 [  1.   0.  10.]
 [  0.   1.   8.]
 [  0.   1.   9.]]


In [25]:
# training a simple regression model
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [26]:
# building a simple linear model with 1 output
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [27]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-0.77906322]
 [-1.55812645]
 [-2.33718967]
 [-3.1162529 ]]


In [28]:
# to optimize a model, you first need to define the loss, MSE, standard loss for regression problem

loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

0.261414


In [29]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [30]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

0.261414
0.227941
0.204436
0.18785
0.176066
0.167616
0.161481
0.156954
0.153544
0.150912
0.14882
0.147104
0.145652
0.144383
0.143244
0.142197
0.141214
0.140277
0.139375
0.138497
0.137638
0.136794
0.135961
0.135138
0.134323
0.133515
0.132713
0.131917
0.131127
0.130342
0.129562
0.128786
0.128016
0.12725
0.126489
0.125733
0.124981
0.124234
0.123491
0.122753
0.122019
0.121289
0.120564
0.119843
0.119127
0.118414
0.117706
0.117003
0.116303
0.115608
0.114917
0.11423
0.113547
0.112868
0.112193
0.111522
0.110855
0.110193
0.109534
0.108879
0.108228
0.107581
0.106938
0.106298
0.105663
0.105031
0.104403
0.103779
0.103158
0.102541
0.101928
0.101319
0.100713
0.100111
0.0995125
0.0989176
0.0983262
0.0977383
0.0971539
0.0965731
0.0959957
0.0954217
0.0948512
0.0942841
0.0937204
0.0931601
0.0926031
0.0920494
0.091499
0.090952
0.0904082
0.0898677
0.0893304
0.0887963
0.0882654
0.0877377
0.0872131
0.0866917
0.0861733
0.0856581
